# DataLoader

In [28]:
##config

DIR_AMAZON = "../office31/amazon"
DIR_DSLR = "../office31/dslr"
DIR_WEBCAM = "../office31/webcam"

BATCH_SIZE = 64
IMG_HEIGHT = 128
IMG_WIDTH = 128
IMG_DIM = 3

EPOCHS = 100

In [29]:
import tensorflow as tf

class DataLoader:
    
    def __init__(self, source, target, BATCH_SIZE=64, IMG_HEIGHT = 128, IMG_WIDTH = 128, IMG_DIM = 3):
        self.source = source
        self.target = target
        self.BATCH_SIZE = BATCH_SIZE
        self.IMG_HEIGHT = IMG_HEIGHT
        self.IMG_WIDTH = IMG_WIDTH
        self.IMG_DIM = IMG_DIM
        self.normalization_layer = tf.keras.layers.Rescaling(1./255)
    
    def load_source_train(self):
        dataset_source =  tf.keras.utils.image_dataset_from_directory(
                                self.source,
                                validation_split=0.2,
                                subset="training",
                                seed=123,
                                image_size=(self.IMG_HEIGHT, self.IMG_WIDTH),
                                batch_size=self.BATCH_SIZE)
        
        return dataset_source.map(lambda x, y: (self.normalization_layer(x), y))
    
    
    def load_source_validation(self):
        dataset_source_val =  tf.keras.utils.image_dataset_from_directory(
                                self.source,
                                validation_split=0.2,
                                subset="validation",
                                seed=123,
                                image_size=(self.IMG_HEIGHT, self.IMG_WIDTH),
                                batch_size=self.BATCH_SIZE)
        
        return dataset_source_val.map(lambda x, y: (self.normalization_layer(x), y))
    
    def load_target_test(self):
        dataset_target =  tf.keras.utils.image_dataset_from_directory(
                                self.target,
                                seed=123,
                                image_size=(self.IMG_HEIGHT, self.IMG_WIDTH),
                                batch_size=self.BATCH_SIZE)
        
        return dataset_target.map(lambda x, y: (self.normalization_layer(x), y))
    
    def load(self):
        return tf.data.Dataset.zip((self.load_source_train(), self.load_source_validation(), self.load_target_test()))
        

In [30]:
dataset = DataLoader(DIR_AMAZON, DIR_WEBCAM).load()

Found 2817 files belonging to 31 classes.
Using 2254 files for training.
Found 2817 files belonging to 31 classes.
Using 563 files for validation.
Found 795 files belonging to 31 classes.


# Model

In [36]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers


class AML(tf.keras.Model):
    """Adeversarial Multitask Learning."""
    
    def __init__(self):
        super(AML, self).__init__()
        
        self.InceptionV3 = tf.keras.applications.InceptionV3(
                            weights='imagenet',
                            input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_DIM),
                            include_top=False)
        
        self.InceptionV3.trainable = False
        
        self.feature_extractor = keras.Sequential(
            [
                self.InceptionV3,
                layers.GlobalAveragePooling2D(),
                layers.Flatten(),
                layers.Dropout(0.5),
                layers.Dense(512, activation='relu', kernel_initializer='he_uniform'),

            ],
            name="feature_extractor",
        )

In [37]:
AML()

2024-01-08 04:34:24.974972: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:735] failed to allocate 13.94M (14620928 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-01-08 04:34:24.976174: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:735] failed to allocate 13.94M (14620928 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-01-08 04:34:34.982073: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:735] failed to allocate 13.94M (14620928 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-01-08 04:34:34.985721: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:735] failed to allocate 13.94M (14620928 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-01-08 04:34:34.985785: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 840.0KiB (rounded to 860160)requested by op AddV2
If the cause is memory fragmentation maybe the env

ResourceExhaustedError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2]